In [14]:
import pandas as pd
import os
import glob


In [ ]:
data_file_path = r'G:\FYP_v2\PrePostMarketDataset\processed_data\*\*.parquet'
ohlcv_folder = r"E:\CampusAcedemics2\SEM7\FYP\Intellitrade\New_data_2\*.csv"

input_files = glob.glob(data_file_path)
output_files = glob.glob(ohlcv_folder)

df_input_list = [pd.read_parquet(file) for file in input_files]
df_output_list = [pd.read_parquet(file) for file in output_files]

df_input = pd.concat(df_input_list, ignore_index=True)
df_output = pd.concat(df_output_list, ignore_index=True)

In [16]:
df_input.tail()

,DATE,EX,SYM_SUFFIX,TR_SCOND,SIZE,PRICE,TR_STOP_IND,TR_CORR,TR_SEQNUM,TR_ID,TR_SOURCE,TR_RF,PART_TIME,TRF_TIME,TTE_IND,SYM_ROOT,TIME_M,Market_Session
6337932,2019-04-30,b'P',b'0',b'9',0.0,150.39,b'N',b'00',41393701.0,b'52983560095051',b'C',b'0',66600.000205,NaN,b'1',VTI,0 days 18:30:00.000758,Post-Market
6337933,2019-04-30,b'P',b'0',b'M',24644.0,150.39,b'N',b'00',41393901.0,b'52983560095052',b'C',b'0',66600.000205,NaN,b'0',VTI,0 days 18:30:00.000854,Post-Market
6337934,2019-04-30,b'D',b'0',b'TI',50.0,150.75,b'N',b'00',41482801.0,b'79372192602678',b'C',b'N',70921.305000,70921.308613,b'0',VTI,0 days 19:42:01.310216,Post-Market
6337935,2019-04-30,b'P',b'0',b'9',0.0,150.39,b'N',b'00',41489701.0,b'52983560097095',b'C',b'0',72000.002349,NaN,b'1',VTI,0 days 20:00:00.003670,Post-Market
6337936,2019-04-30,b'P',b'0',b'M',24644.0,150.39,b'N',b'00',41489901.0,b'52983560097096',b'C',b'0',72000.002349,NaN,b'0',VTI,0 days 20:00:00.003725,Post-Market


In [17]:
df_input.describe()

,DATE,SIZE,PRICE,TR_SEQNUM,PART_TIME,TRF_TIME,TIME_M
count,6337937,6.337937e+06,6.337937e+06,6.337937e+06,6.337937e+06,1.000506e+06,6337937
mean,2019-02-28 18:18:35.498746112,1.011947e+03,4.362099e+02,6.773715e+07,4.875275e+04,5.073541e+04,0 days 13:32:46.767324830
min,2019-01-02 00:00:00,0.000000e+00,1.220000e+01,1.002000e+03,6.000000e+01,2.880002e+04,0 days 04:00:00.001416
25%,2019-01-30 00:00:00,1.000000e+01,1.054325e+02,4.487010e+05,3.420601e+04,3.423342e+04,0 days 09:30:06.440638289
50%,2019-02-28 00:00:00,8.200000e+01,1.538500e+02,2.572149e+06,5.756307e+04,5.757215e+04,0 days 15:59:23.086618079
75%,2019-04-01 00:00:00,1.000000e+02,2.656600e+02,1.672467e+08,5.759683e+04,5.879391e+04,0 days 15:59:56.845669
max,2019-04-30 00:00:00,2.073649e+07,3.251593e+05,2.865459e+08,7.200000e+04,7.199993e+04,0 days 20:00:00.011213
std,NaN,4.774776e+04,3.244062e+03,9.196527e+07,1.324035e+04,1.205782e+04,0 days 03:40:19.033606172


In [18]:
# Convert to correct Data Types

# Convert Date to datetime
df_input["DATE"] = pd.to_datetime(df_input["DATE"])


# Convert Market Session column to string
df_input["Market_Session"] = df_input["Market_Session"].astype(str)



In [19]:
# --- Aggregate Data at the Daily Level ---
daily_df = df_input.groupby(["SYM_ROOT", "DATE"]).agg(
    pre_market_volume=("SIZE", lambda x: x[df_input["Market_Session"] == "Pre-Market"].sum()),
    post_market_volume=("SIZE", lambda x: x[df_input["Market_Session"] == "Post-Market"].sum()),
).reset_index()

In [24]:
# Add 'DAY_OF_WEEK' column
daily_df['DAY_OF_WEEK'] = daily_df['DATE'].dt.day_name()

In [ ]:

daily_df.drop('daily_VWAP', axis=1, inplace=True)

In [30]:
daily_df.tail(50)

,SYM_ROOT,DATE,pre_market_volume,post_market_volume,DAY_OF_WEEK
1836,VTI,2019-02-19,261804.0,138544.0,Tuesday
1837,VTI,2019-02-20,129722.0,191820.0,Wednesday
1838,VTI,2019-02-21,139449.0,135141.0,Thursday
1839,VTI,2019-02-22,83941.0,211287.0,Friday
1840,VTI,2019-02-25,273075.0,138213.0,Monday
1841,VTI,2019-02-26,125232.0,75517.0,Tuesday
1842,VTI,2019-02-27,152398.0,249043.0,Wednesday
1843,VTI,2019-02-28,85249.0,479231.0,Thursday
1844,VTI,2019-03-01,144815.0,85412.0,Friday
1845,VTI,2019-03-04,330311.0,91111.0,Monday


In [ ]:
daily_df[daily_df['SYM_ROOT']=='AYX'].head(50)